In [68]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

In [75]:
# csv_file = os.path.abspath('')
data = np.genfromtxt('./dataR2.csv', delimiter=',', skip_header=True)
# mieszam sobie zbiór, bo... bo mogę
np.random.shuffle(data)
# height to de facto liczba badań
height, _ = data.shape
# liczba zbiorów testowych, liczba zbiorów treningowych
# potem się okaże, że wystarczy jedno z nich, ale nie chce mi się już używać korektora
n_test, n_train = round(0.2*height), height-round(0.2*height)
# sprawdzam, czy nie potknąłem się o zaokrąglenie, ale pewnie jakby wziąć pod uwagę dokładność numeryczną, to poniższy krok prawdopodobnie nie ma sensu
# (sztuka dla sztuki, jak paw dumnie prezentuje swój ogon, tak jak dumnie prezentuję znajomość niesamowicie skomplikowanej składni języka Python)
# <owacje>
assert n_test+n_train==height

In [76]:
# remove broken data (needles if skip_header=True)
filtered = []
for row in data:
    if np.isnan(np.sum(row)):
        print(row)

In [77]:
train_set = data[:n_train,:]
train_x, train_y = train_set[:,:-1], train_set[:,-1:].squeeze()
print(train_x.shape, train_y.shape)
test_set=data[n_train:,:]
test_x, test_y = test_set[:,:-1], test_set[:,-1:].squeeze()
print(train_set.shape, test_set.shape, n_train, n_test)
# nie ma potrzeby normalizacji train_y i test_y do {0,1}, ponieważ macierz predykcji wskazuje na prawdopodobieństwo wystąpienia kolejnych klas
# (w naszym przypadku to 1 i 2)

(93, 9) (93,)
(93, 10) (23, 10) 93 23


In [78]:
model = LogisticRegression(solver='liblinear', random_state=0, ).fit(train_x, train_y)
# podobno liblinear is good for small datasets, a ja nie mam zamiaru się z tym kłucić

In [110]:
print(model.classes_)
# ważne!
# w naszym przypadku są dwie klasy (liczba kolumn w macierzy predict_proba)
# model.predict bierze pod uwagę macierz n x 2, gdzie pierwsza kolumna to prawdopodobieństwo pierwszej klasy (1),
# a druga kolumna to prawd drugiej klasy (2). Czyli (wg mnie) próg odcięcia tutaj wynosi 0.5
pred_y = model.predict(test_x)
cls_report = classification_report(test_y, pred_y, output_dict=False)
cls_report_dict = classification_report(test_y, pred_y, output_dict=True)
print(cls_report)
trafnosc_predykcji = cls_report_dict.keys()
trafnosc_decyzji = round(cls_report_dict['macro avg']['f1-score'], 3)
# podobno recall to jest czułość
czulosc = round(cls_report_dict['macro avg']['recall'], 3)
print('***')
print(f'Odp.: Trafność decyzji wynosi {trafnosc_decyzji}, a czułość wyniosła {czulosc}.')

[1. 2.]
              precision    recall  f1-score   support

         1.0       0.53      0.89      0.67         9
         2.0       0.88      0.50      0.64        14

    accuracy                           0.65        23
   macro avg       0.70      0.69      0.65        23
weighted avg       0.74      0.65      0.65        23

***
Odp.: Trafność decyzji wynosi 0.652, a czułość wyniosła 0.694.
